## Step 1: Set up the environment

First, let's install the required libraries and set up the MultiOn client.

In [2]:
%pip install multion

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: psutil
    Found existing installation: psutil 5.9.5
    Uninstalling psutil-5.9.5:
      Successfully uninstalled psutil-5.9.5
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1


In [49]:

%pip install mem0ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.8/328.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.3/309.3 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.64.1
    Uninstalling grpcio-1.64.1:
      Successfully uninstalled grpcio-1.64.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beha

In [66]:
from multion.client import MultiOn

client = MultiOn(
    api_key="MULTION_API_KEY" # Get your API key from https://app.multion.ai/api-keys
)

In [67]:

import os
from mem0 import Memory

# Configuration
OPENAI_API_KEY = 'OPENAI_API_KEY'  # Replace with your actual OpenAI API key
MULTION_API_KEY = 'MULTION_API_KEY'
USER_ID = "test"

# Set up OpenAI API key
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

# Initialize Mem0 and MultiOn
memory = Memory()
multion = MultiOn(api_key=MULTION_API_KEY)

## Step 2: Create the search agent

Next, we will create an agent session with local mode enabled, which allows us to see the agent in action in our local browser with the MultiOn Chrome Extension. This agent will be responsible for searching and navigating company search results.

Make sure that the MultiOn Chrome Extension is installed and enabled (for more details, see [here](https://docs.multion.ai/learn/browser-extension)).

In [68]:
create_response = client.sessions.create(
    url="https://linkedin.com",
    local=True
)

session_id = create_response.session_id

If you aren't logged in to LinkedIn yet, make sure you do so. Now, we can continue from the same session using the `session_id` from the response. We will search for people related to MultiOn and use `step` until we get to the people results page.

In [69]:
status = "CONTINUE"

linkedin_url = "https://www.linkedin.com/in/alex-reibman-67951589/"

while status == "CONTINUE":
    step_response = client.sessions.step(
        session_id=session_id,
        cmd=f"open {linkedin_url}"
    )
    status = step_response.status

## Step 3: Scrape search results

Once we are on the results page, we can start retrieving data with `retrieve`.

Since LinkedIn paginates its results, we will have to navigate to the next page with `step` once we have scraped all profiles on the current page. Conveniently, `retrieve` has an option to scroll down to the bottom while retrieving (`scroll_to_bottom`), which we will use to speed up the process. We can also enable `render_js` to get all image URLs.

In [70]:
retrieve_response = client.retrieve(
    session_id=session_id,
    cmd="Get name, headline, location, current position, profile URL, and image URL.",
    fields=["name", "headline", "location", "current_position", "profile_url", "image_url"],
    scroll_to_bottom=True,
    render_js=True
)
print(retrieve_response.data[0])

print(f"Scraped page with {len(retrieve_response.data)}")




{'name': 'Alex Reibman', 'headline': 'Co-Founder/CEO @ AgentOps', 'location': 'San Francisco, California, United States', 'current_position': 'Co-Founder', 'profile_url': '#verified-medium', 'image_url': 'https://media.licdn.com/dms/image/C4E03AQHmQW9F92xCyg/profile-displayphoto-shrink_400_400/0/1651643223877?e=1727308800&v=beta&t=3QHNWzMxuxZEJv-3s0PChTcmD_Vf606kY6Ip3qusZY8'}
Scraped page with 11


In [71]:
create_response = client.sessions.create(
    url="https://github.com/areibman",
    local=True
)

session_id = create_response.session_id

In [72]:
retrieve_response = client.retrieve(
    session_id=session_id,
    cmd="Get name, location, number of repositories, count of contributions in the last year, followers and following count.",
    fields=["name", "location", "pulbic_repositories", "last_year_contributions_count", "github_followers_count", "github_following_count"],
    scroll_to_bottom=True,
    render_js=True
)
print(retrieve_response.data)

print(f"Scraped page with {len(retrieve_response.data)}")

[{'name': 'Alex Reibman', 'location': None, 'pulbic_repositories': 57, 'last_year_contributions_count': 1151, 'github_followers_count': 123, 'github_following_count': 8}]
Scraped page with 1


In [73]:
screenshot = client.browse(
    session_id=session_id,
    cmd="github profile",
    url="https://github.com/areibman"
)

client.sessions.screenshot(
    session_id=session_id,
)

SessionsScreenshotResponse(screenshot='https://multion-client-screenshots.s3.us-east-2.amazonaws.com/5bd96572-f874-4b4c-9abd-e81280d63cff_fe230d48-a4c3-454a-8855-00f7e98ff327_screenshot.png')

In [65]:
from multion import SessionsStepStreamRequestBrowserParams
from multion.client import MultiOn

response = client.sessions.step_stream(
    session_id=session_id,
    cmd="github profile",
    url="https://github.com/areibman",
    browser_params=SessionsStepStreamRequestBrowserParams(
        height=1.1,
        width=1.1,
    ),
    temperature=1.1,
    # agent_id="string",
    mode="fast",
    include_screenshot=True,
)

print(response)




<generator object WrappedSessionsClient.step_stream.<locals>.generator at 0x7c10e6d62500>
